In [1]:
import pandas as pd
# load and auto-reload modules
%load_ext autoreload
%autoreload 2


In [2]:
# code to use the src module
import sys
sys.path.append('src')
from src.modules.paths_reference import ROOT_PATH

In [3]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# Cargar el modelo GPT-2 preentrenado y su tokenizador
model_name = "datificate/gpt2-small-spanish" 
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)


In [4]:
def calculate_perplexity(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs["input_ids"])
        loss = outputs.loss
        perplexity = torch.exp(loss)
    return perplexity.item()

In [5]:
df = pd.read_excel(ROOT_PATH/ 'data'/'results.xlsx')

df.columns

Index(['model', 'text_generated'], dtype='object')

In [6]:
for i in range(len(df)):
    text = df.loc[i, 'text_generated']
    perplexity = calculate_perplexity(text, model, tokenizer)
    df.loc[i, 'perplexity'] = perplexity


In [39]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import kaleido

df_grouped = df.groupby('model')['perplexity'].median().reset_index()

# Crear la figura con un subplot
fig = make_subplots(rows=1, cols=1)

# Añadir las barras
rgba_color ='rgba(9, 9, 9, 0.75)'
fig.add_trace(
    go.Bar(
        x=df_grouped['model'],
        y=df_grouped['perplexity'],
        marker_color='rgb(0, 123, 255)',  # Azul sólido sin transparencia
        name='Median Perplexity',
        text=df_grouped['perplexity'].round(1),  # Texto para los marcadores
        textposition='outside',  # Posición del texto
        textfont=dict(color=rgba_color),  # Color negro para el texto de las barras
    )
)
# Actualizar el diseño
fig.update_layout(
    title='Mediana de Perplejidad por Modelo',
    title_font_size=18,
    title_font_color=rgba_color,  # Color negro para el título
    xaxis_title='Model',
    yaxis_title='Perplexity',
    font=dict(family="Arial", size=12, color=rgba_color),  # Color negro para todo el texto
    plot_bgcolor='white',  # Fondo del área de trazado blanco
    paper_bgcolor='white',  # Fondo del papel (área fuera de los ejes) blanco
    xaxis=dict(
        showgrid=False,
        zeroline=False,
        ticktext=['PhI-3', 'RAG - GPT'],  # Nuevos nombres para los modelos
        tickvals=df_grouped['model'],  # Posiciones de los ticks originales
        linecolor='rgba(9, 9, 9, 0.35)',  # Color negro para el eje X
        tickfont=dict(color=rgba_color),  # Color negro para las etiquetas del eje X
    ),
    yaxis=dict(
        showgrid=False,
        zeroline=False,
        linecolor='rgba(9, 9, 9, 0.35)',  # Color negro para el eje Y
        tickfont=dict(color=rgba_color),  # Color negro para las etiquetas del eje Y
    ),
    width=500,  # Ancho en píxeles
    height=400,  # Alto en píxeles
    margin=dict(l=50, r=50, t=50, b=50)  # Márgenes
)

# Mostrar la figura
fig.show()

